In [ ]:
## 필요한 패키지를 불러옵니다.
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv

import base64
from io import BytesIO
import os
import openai
from PIL import Image

load_dotenv()

## 문서를 참고할 수 있는 모델을 만들어줍니다.
model = ChatOpenAI(model="gpt-4o-mini")
qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [ ]:
# 만들어놓은 저장소를 불러와서 여러분이 하게 될 질문과 비슷한 문서를 몇 개 가져올지 정합니다.
def search_documents(query, k=2):
    faiss_data = FAISS.load_local("./faiss_data", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
    retriever = faiss_data.as_retriever(search_type='mmr', search_kwargs={'k': k, 'fetch_k' : 5})
    search_results = retriever.get_relevant_documents(query)
    return [result.page_content for result in search_results]

# 앞서 문서를 참고할 수 있는 모델을 바탕으로 답변을 받아볼 수 있는 함수를 만듭니다.
def get_response(prompt, user_input):
    relevant_docs = search_documents(user_input)
    combined_docs = "\n".join(relevant_docs)
    response = qa_document_chain.run(input_document=combined_docs, question=prompt)
    return response

In [ ]:
## 저장된 저장소를 바탕으로 답변이 잘 나오는지 확인해봅니다.
user_input = input('여러분이 하고 싶은 질문을 입력해주세요. : ')    
initial_content = """너는 선생님이야. 문서를 참고하여 학생의 질문에 친절하게 응답해줘.
사용자 질문은 다음과 같아 : """ ## 바꿔보세요!
full_prompt = initial_content + user_input
response = get_response(full_prompt, user_input)
print(response)

In [ ]:
## 원하는 이미지에 관련해서도 답변이 가능한지 확인해봅니다.
image = Image.open('./image.jpg') ## 원하는 파일 이미지를 바꾸어도 됩니다.
buffered = BytesIO()
image.save(buffered, format="PNG")
image_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')


system_prompt = '''너는 중학교 1학년 수학 선생님이야. 학생의 수학 질문에 존댓말로 답해.
    수식은 latex 문법으로 답변하고, 풀이는 계산식을 꼭 포함해.
    풀이는 전략의 각 단계에 해당하는 풀이로 알려줘야 해.
    예를 들면 풀이 1단계는 전략 1단계에 대한 풀이 방법이야.
    아래 답변 형식을 반드시 꼭! 맞추어 답변해야 해.

    답변 형식 ----------------

    전략 1단계 : ~~ (너의 답변 : 문제를 풀기 위한 전략 1단계)

    풀이 1단계 : ~~ (너의 답변 : 전략 1단계를 바탕으로 나온 문제 풀이)

    전략 2단계 : ~~ (너의 답변 : 문제를 풀기 위한 전략 2단계)

    풀이 2단계 : ~~ (너의 답변 : 전략 2단계를 바탕으로 나온 문제 풀이)

    답 : ~~ (너의 답변 : 문제에 대한 답)
    
    -----------------
    반드시 위 형식에 맞춰서 답변해.'''


# OpenAI 모델에 이미지와 함께 요청
messages = [
    {"role": "system", "content": system_prompt},
    {
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
            {"type": "text", "text": "이 수학 문제를 분석해주세요."} ## 이 곳도 바꿔볼 수 있습니다!
        ]
    }
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=1000
)

print(response.choices[0].message.content)